<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#TSS" data-toc-modified-id="TSS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>TSS</a></span></li></ul></div>

In [1]:
from collections import defaultdict
import warnings
import logging
import gffutils
import pybedtools
import pandas as pd
import copy
import re
from gffutils.pybedtools_integration import tsses

logging.basicConfig(level=logging.INFO)

In [2]:
gencode_gtf = '/home/cmb-panasas2/skchoudh/genomes/C_albicans_SC5314/Assembly22/annotation/C_albicans_SC5314_version_A22-s07-m01-r50_features.encode.gtf'
gencode_gtf_db = '/home/cmb-panasas2/skchoudh/genomes/C_albicans_SC5314/Assembly22/annotation/C_albicans_SC5314_version_A22-s07-m01-r50_features.encode.gtf.db'
prefix = '/home/cmb-panasas2/skchoudh/genomes/C_albicans_SC5314/Assembly22/annotation/C_albicans_SC5314_version_A22-s07-m01-r50_features.encode.gffutils'
chrsizes = '/home/cmb-panasas2/skchoudh/genomes/C_albicans_SC5314/Assembly22/fasta_v50/C_albicans_SC5314_version_A22-s07-m01-r50_chromosomes_clean_records.sizes'


In [3]:
def create_gene_dict(db):
    '''
    Store each feature line db.all_features() as a dict of dicts
    '''
    gene_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for line_no, feature in enumerate(db.all_features()):
        gene_ids = feature.attributes['gene_id']
        feature_type = feature.featuretype
        if feature_type == 'gene':
            if len(gene_ids)!=1:
                logging.warning('Found multiple gene_ids on line {} in gtf'.format(line_no))
                break
            else:
                gene_id = gene_ids[0]
                gene_dict[gene_id]['gene'] = feature
        else:
            transcript_ids = feature.attributes['transcript_id']

            for gene_id in gene_ids:
                for transcript_id in transcript_ids:
                    gene_dict[gene_id][transcript_id][feature_type].append(feature)
    return gene_dict


In [4]:
db = gffutils.create_db(gencode_gtf, dbfn=gencode_gtf_db, 
                        merge_strategy='merge', force=True)

db = gffutils.FeatureDB(gencode_gtf_db)
gene_dict = create_gene_dict(db)


/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/lib/python2.7/site-packages/gffutils/create.py:730: UserWarning: It appears you have a gene feature in your GTF file. You may want to use the `disable_infer_genes` option to speed up database creation
  "It appears you have a gene feature in your GTF "


In [5]:
def get_gene_list(gene_dict):
    return list(set(gene_dict.keys()))

def get_UTR_regions(gene_dict, gene_id, transcript, cds):
    if len(cds)==0:
        return [], []
    utr5_regions = []
    utr3_regions = []
    utrs = gene_dict[gene_id][transcript]['UTR']
    first_cds = cds[0]
    last_cds = cds[-1]
    for utr in utrs:
        ## Push all cds at once
        ## Sort later to remove duplicates
        strand = utr.strand
        if strand == '+':
            if utr.stop < first_cds.start:
                utr.feature_type = 'five_prime_UTR'
                utr5_regions.append(utr)
            elif utr.start > last_cds.stop:
                utr.feature_type = 'three_prime_UTR'
                utr3_regions.append(utr)
            else:
                #raise RuntimeError('Error with cds: {}\t {} \t {}'.format(utr, last_cds, first_cds))
                print('Error with cds: {}\t {} \t {}'.format(utr, last_cds, first_cds))
        elif strand == '-':
            if utr.stop < first_cds.start:
                utr.feature_type = 'three_prime_UTR'
                utr3_regions.append(utr)
            elif utr.start > last_cds.stop:
                utr.feature_type = 'five_prime_UTR'
                utr5_regions.append(utr)                
            else:
                #raise RuntimeError('Error with cds')    
                print('Error with cds: {}\t {} \t {}'.format(utr, last_cds, first_cds))
    return utr5_regions, utr3_regions
    
def create_bed(regions, bedtype='0'):
    '''Create bed from list of regions
    bedtype: 0 or 1
        0-Based or 1-based coordinate of the BED
    '''
    bedstr = ''
    for region in regions:
        assert len(region.attributes['gene_id']) == 1
        ## GTF start is 1-based, so shift by one while writing 
        ## to 0-based BED format
        if bedtype == '0':
            start = region.start - 1
        else:
            start = region.start
        bedstr += '{}\t{}\t{}\t{}\t{}\t{}\n'.format(region.chrom,
                                             start,
                                             region.stop,
                                             re.sub('\.\d+', '', region.attributes['gene_id'][0]),
                                             '.',
                                             region.strand)
    return bedstr

def rename_regions(regions, gene_id):
    regions = list(regions)
    if len(regions) == 0:
        return []
    for region in regions:
        region.attributes['gene_id'] = gene_id
    return regions

def merge_regions(db, regions):
    if len(regions) == 0:
        return []
    merged = db.merge(sorted(list(regions), key=lambda x: x.start))
    return merged

def merge_regions_nostrand(db, regions):
    if len(regions) == 0:
        return []
    merged = db.merge(sorted(list(regions), key=lambda x: x.start), ignore_strand=True)
    return merged

In [6]:
utr5_bed = ''
utr3_bed = ''
gene_bed = ''
exon_bed = ''
intron_bed = ''
start_codon_bed = ''
stop_codon_bed = ''
cds_bed = ''

gene_list = []

for gene_id in get_gene_list(gene_dict):
    gene_list.append(gene_dict[gene_id]['gene'])
    
    utr5_regions, utr3_regions = [], []
    exon_regions, intron_regions = [], []
    star_codon_regions, stop_codon_regions = [], []
    cds_regions = []
    
    for feature in gene_dict[gene_id].keys():
        if feature == 'gene':
            continue
        cds = list(gene_dict[gene_id][feature]['CDS'])
        exons = list(gene_dict[gene_id][feature]['exon'])
        merged_exons = merge_regions(db, exons)
        introns = db.interfeatures(merged_exons)
        utr5_region, utr3_region = get_UTR_regions(gene_dict, gene_id, feature, cds)
        utr5_regions += utr5_region
        utr3_regions += utr3_region
        exon_regions += exons
        intron_regions += introns
        cds_regions += cds
        
    merged_utr5 = merge_regions(db, utr5_regions)
    renamed_utr5 = rename_regions(merged_utr5, gene_id)
    
    merged_utr3 = merge_regions(db, utr3_regions)
    renamed_utr3 = rename_regions(merged_utr3, gene_id)
    
    merged_exons = merge_regions(db, exon_regions)
    renamed_exons = rename_regions(merged_exons, gene_id)
    
    merged_introns = merge_regions(db, intron_regions)
    renamed_introns = rename_regions(merged_introns, gene_id)
    
    merged_cds = merge_regions(db, cds_regions)
    renamed_cds = rename_regions(merged_cds, gene_id)
    
    utr3_bed += create_bed(renamed_utr3)
    utr5_bed += create_bed(renamed_utr5)
    exon_bed += create_bed(renamed_exons)
    intron_bed += create_bed(renamed_introns)
    cds_bed += create_bed(renamed_cds)
    
    
gene_bed = create_bed(gene_list)
gene_bedtool = pybedtools.BedTool(gene_bed, from_string=True)
utr5_bedtool = pybedtools.BedTool(utr5_bed, from_string=True)
utr3_bedtool = pybedtools.BedTool(utr3_bed, from_string=True)
exon_bedtool = pybedtools.BedTool(exon_bed, from_string=True)
intron_bedtool = pybedtools.BedTool(intron_bed, from_string=True)
cds_bedtool = pybedtools.BedTool(cds_bed, from_string=True)

gene_bedtool.remove_invalid().sort().saveas('{}.genes.bed'.format(prefix))
utr5_bedtool.remove_invalid().sort().saveas('{}.UTR5.bed'.format(prefix))
utr3_bedtool.remove_invalid().sort().saveas('{}.UTR3.bed'.format(prefix))
exon_bedtool.remove_invalid().sort().saveas('{}.exon.bed'.format(prefix))
intron_bedtool.remove_invalid().sort().saveas('{}.intron.bed'.format(prefix))
cds_bedtool.remove_invalid().sort().saveas('{}.cds.bed'.format(prefix))


Error with cds: Ca22chr6B_C_albicans_SC5314	CGD	UTR	356825	356899	.	+	1	 ""; gene_status "NOVEL"; level "1"; transcript_name "C6_01700W_B-T"; transcript_type "protein_coding"; gene_id "C6_01700W_B"; exon_id "C6_01700W_B-T"; exon_number "1"; ccdsid "CAL0000192067"; transcript_id "C6_01700W_B-T"; gene_type "protein_coding"; transcript_status "NOVEL"; gene_name "ACC1"	 Ca22chr6B_C_albicans_SC5314	CGD	CDS	357342	357733	.	+	1	 ""; gene_status "NOVEL"; level "1"; transcript_name "C6_01700W_B-T"; transcript_type "protein_coding"; gene_id "C6_01700W_B"; exon_id "C6_01700W_B-T"; exon_number "2"; ccdsid "CAL0000192067"; transcript_id "C6_01700W_B-T"; gene_type "protein_coding"; transcript_status "NOVEL"; gene_name "ACC1" 	 Ca22chr6B_C_albicans_SC5314	CGD	CDS	356899	356899	.	+	1	 ""; gene_status "NOVEL"; level "1"; transcript_name "C6_01700W_B-T"; transcript_type "protein_coding"; gene_id "C6_01700W_B"; exon_id "C6_01700W_B-T"; exon_number "1"; ccdsid "CAL0000192067"; transcript_id "C6_01700W_B-T

Error with cds: Ca22chrRA_C_albicans_SC5314	CGD	UTR	743076	743077	.	-	1	 ""; gene_status "NOVEL"; exon_number "2"; level "1"; transcript_type "protein_coding"; gene_id "CR_03310C_A"; exon_id "CR_03310C_A-T"; transcript_name "CR_03310C_A-T"; ccdsid "CAL0000182324"; transcript_status "NOVEL"; transcript_id "CR_03310C_A-T"; gene_type "protein_coding"; protein_id "orf19.2393,orf19.9929"; gene_name "SAP6"	 Ca22chrRA_C_albicans_SC5314	CGD	CDS	743039	743076	.	-	1	 ""; gene_status "NOVEL"; exon_number "2"; level "1"; transcript_type "protein_coding"; gene_id "CR_03310C_A"; exon_id "CR_03310C_A-T"; transcript_name "CR_03310C_A-T"; ccdsid "CAL0000182324"; transcript_status "NOVEL"; transcript_id "CR_03310C_A-T"; gene_type "protein_coding"; protein_id "orf19.2393,orf19.9929"; gene_name "SAP6" 	 Ca22chrRA_C_albicans_SC5314	CGD	CDS	742233	742936	.	-	1	 ""; gene_status "NOVEL"; exon_number "1"; level "1"; transcript_type "protein_coding"; gene_id "CR_03310C_A"; exon_id "CR_03310C_A-T"; transcript_na

Error with cds: Ca22chr3B_C_albicans_SC5314	CGD	UTR	940780	941259	.	-	.	 ""; gene_status "NOVEL"; level "1"; transcript_name "C3_04500C_B-T"; transcript_type "protein_coding"; gene_id "C3_04500C_B"; exon_id "C3_04500C_B-T"; exon_number "1"; ccdsid "CAL0000197042"; transcript_id "C3_04500C_B-T"; gene_type "protein_coding"; transcript_status "NOVEL"; gene_name "C1_05660C_B"	 Ca22chr3B_C_albicans_SC5314	CGD	CDS	942263	942264	.	-	1	 ""; gene_status "NOVEL"; level "1"; transcript_name "C3_04500C_B-T"; transcript_type "protein_coding"; gene_id "C3_04500C_B"; exon_id "C3_04500C_B-T"; exon_number "2"; ccdsid "CAL0000197042"; transcript_id "C3_04500C_B-T"; gene_type "protein_coding"; transcript_status "NOVEL"; gene_name "C1_05660C_B" 	 Ca22chr3B_C_albicans_SC5314	CGD	CDS	941259	941826	.	-	1	 ""; gene_status "NOVEL"; level "1"; transcript_name "C3_04500C_B-T"; transcript_type "protein_coding"; gene_id "C3_04500C_B"; exon_id "C3_04500C_B-T"; exon_number "1"; ccdsid "CAL0000197042"; transcript_id 

<BedTool(/home/cmb-panasas2/skchoudh/genomes/C_albicans_SC5314/Assembly22/annotation/C_albicans_SC5314_version_A22-s07-m01-r50_features.encode.gffutils.cds.bed)>

In [7]:
for gene_id in get_gene_list(gene_dict):
    start_codons = []
    stop_codons = []
    for start_codon in db.children(gene_id, featuretype='start_codon'):
        ## 1 -based stop
        ## 0-based start handled while converting to bed
        start_codon.stop = start_codon.start
        start_codons.append(start_codon)
    for stop_codon in db.children(gene_id, featuretype='stop_codon'):
        stop_codon.start = stop_codon.stop
        stop_codon.stop = stop_codon.stop+1
        stop_codons.append(stop_codon)
    merged_start_codons = merge_regions(db, start_codons)
    renamed_start_codons = rename_regions(merged_start_codons, gene_id)
    merged_stop_codons = merge_regions(db, stop_codons)
    renamed_stop_codons = rename_regions(merged_stop_codons, gene_id)
    
    start_codon_bed += create_bed(renamed_start_codons)    
    stop_codon_bed += create_bed(renamed_stop_codons)

    
start_codon_bedtool = pybedtools.BedTool(start_codon_bed, from_string=True)
stop_codon_bedtool = pybedtools.BedTool(stop_codon_bed, from_string=True)
start_codon_bedtool.remove_invalid().sort().saveas('{}.start_codon.bed'.format(prefix))
stop_codon_bedtool.remove_invalid().sort().saveas('{}.stop_codon.bed'.format(prefix))

<BedTool(/home/cmb-panasas2/skchoudh/genomes/C_albicans_SC5314/Assembly22/annotation/C_albicans_SC5314_version_A22-s07-m01-r50_features.encode.gffutils.stop_codon.bed)>

In [8]:
## TSS
polyA_sites_bed = ''
tss_sites_bed = ''
for gene_id in get_gene_list(gene_dict):
    tss_sites = []
    polyA_sites = []
    for transcript in db.children(gene_id, featuretype='transcript'):
        start_t = copy.deepcopy(transcript)
        stop_t = copy.deepcopy(transcript)
        
        start_t.stop = start_t.start + 1
        
        stop_t.start = stop_t.stop
        
        if transcript.strand == '-':
            start_t, stop_t = stop_t, start_t
        polyA_sites.append(start_t)
        tss_sites.append(stop_t)
    merged_polyA_sites = merge_regions(db, polyA_sites)
    renamed_polyA_sites = rename_regions(merged_polyA_sites, gene_id)    
    merged_tss_sites = merge_regions(db, tss_sites)
    renamed_tss_sites = rename_regions(merged_tss_sites, gene_id)
    polyA_sites_bed += create_bed(renamed_polyA_sites)    
    tss_sites_bed += create_bed(renamed_tss_sites)

polyA_sites_bedtool = pybedtools.BedTool(polyA_sites_bed, from_string=True)
tss_sites_bedtool = pybedtools.BedTool(tss_sites_bed, from_string=True)
polyA_sites_bedtool.remove_invalid().sort().saveas('{}.polyA_sites.bed'.format(prefix))
tss_sites_bedtool.remove_invalid().sort().saveas('{}.tss_sites.bed'.format(prefix))

<BedTool(/home/cmb-panasas2/skchoudh/genomes/C_albicans_SC5314/Assembly22/annotation/C_albicans_SC5314_version_A22-s07-m01-r50_features.encode.gffutils.tss_sites.bed)>

# TSS

In [9]:
tss = tsses(db, as_bed6=True, merge_overlapping=False)
tss.remove_invalid().sort().saveas('{}.tss_temp.bed'.format(prefix))
promoter = tss.slop(l=1000, r=1000, s=True, g=chrsizes)
promoter.remove_invalid().sort().saveas('{}.promoter.1000.bed'.format(prefix))

<BedTool(/home/cmb-panasas2/skchoudh/genomes/C_albicans_SC5314/Assembly22/annotation/C_albicans_SC5314_version_A22-s07-m01-r50_features.encode.gffutils.promoter.1000.bed)>

In [10]:
for l in [1000, 2000, 3000, 4000, 5000]:
    promoter = tss.slop(l=l, r=l, s=True, g=chrsizes)
    promoter.remove_invalid().sort().saveas('{}.promoter.{}.bed'.format(prefix, l))

In [11]:
for x in db.featuretypes():
    print(x)

CDS
UTR
exon
gene
start_codon
stop_codon
transcript


In [12]:
for gene_id in get_gene_list(gene_dict):
    for transcript in db.children(gene_id, featuretype='transcript'):
        print(transcript.attributes)

transcript_id: ['C3_01970C_A-T']
gene_id: ['C3_01970C_A']
transcript_id: ['C6_03940C_B-T']
gene_id: ['C6_03940C_B']
transcript_id: ['C6_03940C_A-T']
gene_id: ['C6_03940C_A']
transcript_id: ['C3_01970C_B-T']
gene_id: ['C3_01970C_B']
transcript_id: ['C1_14430C_B-T']
gene_id: ['C1_14430C_B']
transcript_id: ['C2_02200W_B-T']
gene_id: ['C2_02200W_B']
transcript_id: ['C2_02200W_A-T']
gene_id: ['C2_02200W_A']
transcript_id: ['C1_14430C_A-T']
gene_id: ['C1_14430C_A']
transcript_id: ['C6_04140C_A-T']
gene_id: ['C6_04140C_A']
transcript_id: ['C6_04140C_B-T']
gene_id: ['C6_04140C_B']
transcript_id: ['C1_13730C_A-T']
gene_id: ['C1_13730C_A']
transcript_id: ['C1_13730C_B-T']
gene_id: ['C1_13730C_B']
transcript_id: ['C1_13590W_A-T']
gene_id: ['C1_13590W_A']
transcript_id: ['C1_13590W_B-T']
gene_id: ['C1_13590W_B']
transcript_id: ['C7_00150W_B-T']
gene_id: ['C7_00150W_B']
transcript_id: ['C3_01510W_B-T']
gene_id: ['C3_01510W_B']
transcript_id: ['CR_02650C_B-T']
gene_id: ['CR_02650C_B']
transcript_id:

transcript_id: ['C2_10710W_B-T']
gene_id: ['C2_10710W_B']
transcript_id: ['CR_01320C_B-T']
gene_id: ['CR_01320C_B']
transcript_id: ['C1_07350C_A-T']
gene_id: ['C1_07350C_A']
transcript_id: ['C1_07350C_B-T']
gene_id: ['C1_07350C_B']
transcript_id: ['C5_04810W_B-T']
gene_id: ['C5_04810W_B']
transcript_id: ['C5_04810W_A-T']
gene_id: ['C5_04810W_A']
transcript_id: ['C1_02190W_B-T']
gene_id: ['C1_02190W_B']
transcript_id: ['C6_00210W_A-T']
gene_id: ['C6_00210W_A']
transcript_id: ['C6_00210W_B-T']
gene_id: ['C6_00210W_B']
transcript_id: ['C1_02190W_A-T']
gene_id: ['C1_02190W_A']
transcript_id: ['C1_07440W_A-T']
gene_id: ['C1_07440W_A']
transcript_id: ['C1_07440W_B-T']
gene_id: ['C1_07440W_B']
transcript_id: ['C2_07710W_A-T']
gene_id: ['C2_07710W_A']
transcript_id: ['C2_07710W_B-T']
gene_id: ['C2_07710W_B']
transcript_id: ['C1_05650W_A-T']
gene_id: ['C1_05650W_A']
transcript_id: ['C1_05650W_B-T']
gene_id: ['C1_05650W_B']
transcript_id: ['C6_01050W_B-T']
gene_id: ['C6_01050W_B']
transcript_id:

transcript_id: ['C6_01490C_A-T']
gene_id: ['C6_01490C_A']
transcript_id: ['C6_00240C_B-T']
gene_id: ['C6_00240C_B']
transcript_id: ['C6_04470C_A-T']
gene_id: ['C6_04470C_A']
transcript_id: ['C6_04470C_B-T']
gene_id: ['C6_04470C_B']
transcript_id: ['CR_07770C_B-T']
gene_id: ['CR_07770C_B']
transcript_id: ['CR_07770C_A-T']
gene_id: ['CR_07770C_A']
transcript_id: ['C7_00260C_A-T']
gene_id: ['C7_00260C_A']
transcript_id: ['C7_00260C_B-T']
gene_id: ['C7_00260C_B']
transcript_id: ['C4_01990W_B-T']
gene_id: ['C4_01990W_B']
transcript_id: ['C4_01990W_A-T']
gene_id: ['C4_01990W_A']
transcript_id: ['C1_03350C_A-T']
gene_id: ['C1_03350C_A']
transcript_id: ['C1_03350C_B-T']
gene_id: ['C1_03350C_B']
transcript_id: ['C1_06840C_A-T']
gene_id: ['C1_06840C_A']
transcript_id: ['CR_00780C_B-T']
gene_id: ['CR_00780C_B']
transcript_id: ['C5_02710W_A-T']
gene_id: ['C5_02710W_A']
transcript_id: ['C5_02710W_B-T']
gene_id: ['C5_02710W_B']
transcript_id: ['CR_00780C_A-T']
gene_id: ['CR_00780C_A']
transcript_id:

transcript_id: ['C1_10220C_A-T']
gene_id: ['C1_10220C_A']
transcript_id: ['C2_00100C_A-T']
gene_id: ['C2_00100C_A']
transcript_id: ['C2_00100C_B-T']
gene_id: ['C2_00100C_B']
transcript_id: ['C1_10220C_B-T']
gene_id: ['C1_10220C_B']
transcript_id: ['C5_03760C_B-T']
gene_id: ['C5_03760C_B']
transcript_id: ['C5_03760C_A-T']
gene_id: ['C5_03760C_A']
transcript_id: ['C7_02480W_B-T']
gene_id: ['C7_02480W_B']
transcript_id: ['CR_00660W_B-T']
gene_id: ['CR_00660W_B']
transcript_id: ['CR_00660W_A-T']
gene_id: ['CR_00660W_A']
transcript_id: ['C7_02480W_A-T']
gene_id: ['C7_02480W_A']
transcript_id: ['C3_05060W_B-T']
gene_id: ['C3_05060W_B']
transcript_id: ['C3_05060W_A-T']
gene_id: ['C3_05060W_A']
transcript_id: ['C2_05290C_A-T']
gene_id: ['C2_05290C_A']
transcript_id: ['C2_02420C_B-T']
gene_id: ['C2_02420C_B']
transcript_id: ['C2_02420C_A-T']
gene_id: ['C2_02420C_A']
transcript_id: ['C7_02260W_B-T']
gene_id: ['C7_02260W_B']
transcript_id: ['C1_08350C_B-T']
gene_id: ['C1_08350C_B']
transcript_id:

transcript_id: ['C2_00350W_A-T']
gene_id: ['C2_00350W_A']
transcript_id: ['C1_03840W_B-T']
gene_id: ['C1_03840W_B']
transcript_id: ['C4_06110C_A-T']
gene_id: ['C4_06110C_A']
transcript_id: ['C4_06110C_B-T']
gene_id: ['C4_06110C_B']
transcript_id: ['C1_10040W_A-T']
gene_id: ['C1_10040W_A']
transcript_id: ['C1_10040W_B-T']
gene_id: ['C1_10040W_B']
transcript_id: ['CR_05970C_B-T']
gene_id: ['CR_05970C_B']
transcript_id: ['C1_02400C_A-T']
gene_id: ['C1_02400C_A']
transcript_id: ['C1_02400C_B-T']
gene_id: ['C1_02400C_B']
transcript_id: ['CR_05970C_A-T']
gene_id: ['CR_05970C_A']
transcript_id: ['CR_09890C_B-T']
gene_id: ['CR_09890C_B']
transcript_id: ['CR_09890C_A-T']
gene_id: ['CR_09890C_A']
transcript_id: ['C5_00040C_B-T']
gene_id: ['C5_00040C_B']
transcript_id: ['C4_02140C_A-T']
gene_id: ['C4_02140C_A']
transcript_id: ['C4_02140C_B-T']
gene_id: ['C4_02140C_B']
transcript_id: ['C5_00040C_A-T']
gene_id: ['C5_00040C_A']
transcript_id: ['C1_10290W_B-T']
gene_id: ['C1_10290W_B']
transcript_id:

...

transcript_id: ['C2_05690C_A-T']
gene_id: ['C2_05690C_A']
transcript_id: ['C6_04050W_A-T']
gene_id: ['C6_04050W_A']
transcript_id: ['C5_00370W_A-T']
gene_id: ['C5_00370W_A']
transcript_id: ['C7_03680W_A-T']
gene_id: ['C7_03680W_A']
transcript_id: ['C7_03680W_B-T']
gene_id: ['C7_03680W_B']
transcript_id: ['C5_00370W_B-T']
gene_id: ['C5_00370W_B']
transcript_id: ['C1_03960C_A-T']
gene_id: ['C1_03960C_A']
transcript_id: ['C1_03960C_B-T']
gene_id: ['C1_03960C_B']
transcript_id: ['C3_05170W_B-T']
gene_id: ['C3_05170W_B']
transcript_id: ['C1_06100C_B-T']
gene_id: ['C1_06100C_B']
transcript_id: ['C1_06100C_A-T']
gene_id: ['C1_06100C_A']
transcript_id: ['C7_01280C_B-T']
gene_id: ['C7_01280C_B']
transcript_id: ['C2_01880C_B-T']
gene_id: ['C2_01880C_B']
transcript_id: ['C2_01880C_A-T']
gene_id: ['C2_01880C_A']
transcript_id: ['C1_01390C_B-T']
gene_id: ['C1_01390C_B']
transcript_id: ['C1_01390C_A-T']
gene_id: ['C1_01390C_A']
transcript_id: ['C2_03080W_A-T']
gene_id: ['C2_03080W_A']
transcript_id:

transcript_id: ['C4_02180C_A-T']
gene_id: ['C4_02180C_A']
transcript_id: ['C4_01200C_B-T']
gene_id: ['C4_01200C_B']
transcript_id: ['C3_01040C_B-T']
gene_id: ['C3_01040C_B']
transcript_id: ['C4_03790W_B-T']
gene_id: ['C4_03790W_B']
transcript_id: ['C5_00490C_B-T']
gene_id: ['C5_00490C_B']
transcript_id: ['C5_00490C_A-T']
gene_id: ['C5_00490C_A']
transcript_id: ['C4_03790W_A-T']
gene_id: ['C4_03790W_A']
transcript_id: ['C6_00590W_A-T']
gene_id: ['C6_00590W_A']
transcript_id: ['C6_00590W_B-T']
gene_id: ['C6_00590W_B']
transcript_id: ['C6_04340W_B-T']
gene_id: ['C6_04340W_B']
transcript_id: ['C5_03500W_A-T']
gene_id: ['C5_03500W_A']
transcript_id: ['C1_03900W_B-T']
gene_id: ['C1_03900W_B']
transcript_id: ['C2_06950C_A-T']
gene_id: ['C2_06950C_A']
transcript_id: ['C1_03900W_A-T']
gene_id: ['C1_03900W_A']
transcript_id: ['C1_02640C_B-T']
gene_id: ['C1_02640C_B']
transcript_id: ['CR_09490W_A-T']
gene_id: ['CR_09490W_A']
transcript_id: ['C3_06440W_B-T']
gene_id: ['C3_06440W_B']
transcript_id:

transcript_id: ['C2_09140C_A-T']
gene_id: ['C2_09140C_A']
transcript_id: ['C2_09140C_B-T']
gene_id: ['C2_09140C_B']
transcript_id: ['C5_02280C_B-T']
gene_id: ['C5_02280C_B']
transcript_id: ['C5_02280C_A-T']
gene_id: ['C5_02280C_A']
transcript_id: ['C4_02960W_B-T']
gene_id: ['C4_02960W_B']
transcript_id: ['C2_03630W_B-T']
gene_id: ['C2_03630W_B']
transcript_id: ['C1_05770C_A-T']
gene_id: ['C1_05770C_A']
transcript_id: ['C1_05770C_B-T']
gene_id: ['C1_05770C_B']
transcript_id: ['C4_06130W_B-T']
gene_id: ['C4_06130W_B']
transcript_id: ['CR_07140C_A-T']
gene_id: ['CR_07140C_A']
transcript_id: ['CR_01420W_A-T']
gene_id: ['CR_01420W_A']
transcript_id: ['CR_01420W_B-T']
gene_id: ['CR_01420W_B']
transcript_id: ['CR_00270C_B-T']
gene_id: ['CR_00270C_B']
transcript_id: ['C2_02590W_B-T']
gene_id: ['C2_02590W_B']
transcript_id: ['C2_02590W_A-T']
gene_id: ['C2_02590W_A']
transcript_id: ['CR_00270C_A-T']
gene_id: ['CR_00270C_A']
transcript_id: ['C1_03810C_A-T']
gene_id: ['C1_03810C_A']
transcript_id:

transcript_id: ['C6_01260W_B-T']
gene_id: ['C6_01260W_B']
transcript_id: ['C6_01260W_A-T']
gene_id: ['C6_01260W_A']
transcript_id: ['C3_03110W_A-T']
gene_id: ['C3_03110W_A']
transcript_id: ['C4_01750C_B-T']
gene_id: ['C4_01750C_B']
transcript_id: ['C6_01870C_B-T']
gene_id: ['C6_01870C_B']
transcript_id: ['C1_04500W_A-T']
gene_id: ['C1_04500W_A']
transcript_id: ['C1_04500W_B-T']
gene_id: ['C1_04500W_B']
transcript_id: ['C6_00790C_B-T']
gene_id: ['C6_00790C_B']
transcript_id: ['C6_00790C_A-T']
gene_id: ['C6_00790C_A']
transcript_id: ['C1_13120C_B-T']
gene_id: ['C1_13120C_B']
transcript_id: ['C1_00050C_B-T']
gene_id: ['C1_00050C_B']
transcript_id: ['C1_00050C_A-T']
gene_id: ['C1_00050C_A']
transcript_id: ['CR_08020C_A-T']
gene_id: ['CR_08020C_A']
transcript_id: ['C3_01780C_A-T']
gene_id: ['C3_01780C_A']
transcript_id: ['C3_01780C_B-T']
gene_id: ['C3_01780C_B']
transcript_id: ['CR_08020C_B-T']
gene_id: ['CR_08020C_B']
transcript_id: ['C2_02190C_B-T']
gene_id: ['C2_02190C_B']
transcript_id:

transcript_id: ['C3_04540C_B-T']
gene_id: ['C3_04540C_B']
transcript_id: ['C4_00860C_B-T']
gene_id: ['C4_00860C_B']
transcript_id: ['C1_09250W_B-T']
gene_id: ['C1_09250W_B']
transcript_id: ['C2_06510W_B-T']
gene_id: ['C2_06510W_B']
transcript_id: ['C2_06510W_A-T']
gene_id: ['C2_06510W_A']
transcript_id: ['C6_01340C_A-T']
gene_id: ['C6_01340C_A']
transcript_id: ['C5_02080C_A-T']
gene_id: ['C5_02080C_A']
transcript_id: ['C5_02080C_B-T']
gene_id: ['C5_02080C_B']
transcript_id: ['C1_07250C_B-T']
gene_id: ['C1_07250C_B']
transcript_id: ['C3_07930C_B-T']
gene_id: ['C3_07930C_B']
transcript_id: ['C1_03170C_A-T']
gene_id: ['C1_03170C_A']
transcript_id: ['C3_04540C_A-T']
gene_id: ['C3_04540C_A']
transcript_id: ['C3_07930C_A-T']
gene_id: ['C3_07930C_A']
transcript_id: ['C6_03370W_B-T']
gene_id: ['C6_03370W_B']
transcript_id: ['CR_08350W_B-T']
gene_id: ['CR_08350W_B']
transcript_id: ['C1_09450C_B-T']
gene_id: ['C1_09450C_B']
transcript_id: ['C1_09150W_B-T']
gene_id: ['C1_09150W_B']
transcript_id:

#### 